<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>

<font color='blue'>
<br />
Антон, рад приветствовать тебя! <br />
<br />    

<br />
Меня зовут Николай. <br />И на этом этапе твоего движения к новой профессии от меня будут комментарии по написанному коду. <br />Чтобы меньше "мусорить эфир" и чтобы максимально наглядно отделяться от основного кода, есть предложение условиться в некоторых обозначениях:<br /> <br />
<u><b>ТАКОЙ ШРИФТ</b></u> - всегда начало комментария <br />
<font color='green'>такой шрифт</font> - комментарии о том, что всё ОК <br/>
<font color='orange'>такой шрифт</font> - комментарии о том, что всё ОК по результату,<br> однако есть на что обратить внимание в плане применения техник, или есть способы сделать более короткий или быстрый код <br/>
<font color='red'>такой шрифт</font> - комментарии о том, что есть критичный момент, влияющий на бизнес-результат проекта.<br/>
</font><br /><br />

# Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

#### Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

#### Инструкция по выполнению проекта
- Загрузите и подготовьте данные. Поясните порядок действий.
- Обучите и проверьте модель для каждого региона:

2.1. Разбейте данные на обучающую и валидационную выборки в соотношении 75:25

2.2. Обучите модель и сделайте предсказания на валидационной выборке

2.3. Сохраните предсказания и правильные ответы на валидационной выборке

2.4. Напечатайте на экране средний запас предсказанного сырья и RMSE модели

2.5. Проанализируйте результаты

- Подготовьтесь к расчёту прибыли:

3.1. Все ключевые значения для расчётов сохраните в отдельных переменных

3.2. Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе

3.3. Напишите выводы по этапу подготовки расчёта прибыли

- Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

4.1. Выберите скважины с максимальными значениями предсказаний. Количество скважин зависит от бюджета компании и стоимости разработки одной скважины

4.2. Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям

4.3. Рассчитайте прибыль для полученного объёма сырья

- Посчитайте риски и прибыль для каждого региона:

5.1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли

5.2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль

5.3. Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор

#### Описание данных:

- Данные геологоразведки трёх регионов находятся в файлах: geo_data_0.csv, geo_data_1.csv и geo_data_2.csv.
- id — уникальный идентификатор месторождения;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в месторождении (тыс. баррелей)

#### Условия задачи:

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона проводится исследование 500 точек.
- Бюджет на разработку месторождений — 10 млрд рублей, стоимость бурения одной скважины — 50 млн рублей.
- Один баррель сырья приносит 4500 рублей прибыли.
- Не рассматривать регионы, в которых риск убытков выше 2.5%. Из оставшихся выбирается регион с наибольшей средней прибылью.


# 1. Загрузка и подготовка данных

Для начала импортируем библиотеки, получим все данные и рассмотрим общую информацию об импортированных данных.

In [1]:
import numpy as np
import pandas as pd
from scipy import stats as st
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split, cross_val_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm
import os

geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [2]:
geo_data_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [3]:
geo_data_0 = geo_data_0.drop(columns=['id'])

In [4]:
geo_data_0.head()

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


In [5]:
geo_data_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [6]:
geo_data_1 = geo_data_1.drop(columns=['id'])

In [7]:
geo_data_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [8]:
geo_data_2 = geo_data_2.drop(columns=['id'])

In [9]:
geo_data_0.shape

(100000, 4)

In [10]:
geo_data_1.shape

(100000, 4)

In [11]:
geo_data_2.shape

(100000, 4)

Зафиксируем псевдослучайность для всех используемых в проекте алгоритмов:

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
ОК! лёгкое аналитическое начало)<br>
    

In [12]:
rnd_state = 12345

### Выводы:

- Представленные данные состоят из 4 признаков и 100000 объектов. Целевым признаком, в данном случае, является Product — объём запасов в месторождении (тыс. баррелей) 

- Удалили ненужный столбец id для дальнейшего решение задачи

- Зафиксировали псевдослучайность для всех используемых в проекте алгоритмов

# 2. Обучение и проверка модели

### Рассмотрим регион 1 (geo_data_0):

- обучим модель Линейной регрессии на первом регионе 

- Проверим MSE, R2, RMSE

In [13]:
#Для первого региона geo_0

target_0 = geo_data_0['product']
features_0 = geo_data_0.drop('product', axis=1)
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size=0.25, random_state=rnd_state)

model = LinearRegression()
model.fit(features_train_0, target_train_0)
predicted_valid_0 = model.predict(features_valid_0)
r2 = r2_score(target_valid_0, predicted_valid_0)
mse = mean_squared_error(target_valid_0, predicted_valid_0)
print("MSE первого региона:", mse)
print("R2 первого региона:", r2)
print("RMSE первого региона:", mse ** 0.5)

MSE первого региона: 1412.2129364399243
R2 первого региона: 0.27994321524487786
RMSE первого региона: 37.5794217150813


In [14]:
#Для первого региона geo_1

target_1 = geo_data_1['product']
features_1 = geo_data_1.drop('product', axis=1)
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=rnd_state)

model = LinearRegression()
model.fit(features_train_1, target_train_1)
predicted_valid_1 = model.predict(features_valid_1)
r2 = r2_score(target_valid_1, predicted_valid_1)
mse = mean_squared_error(target_valid_1, predicted_valid_1)
print("MSE второго региона:", mse)
print("R2 второго региона:", r2)
print("RMSE второго региона:", mse ** 0.5)

MSE второго региона: 0.7976263360391139
R2 второго региона: 0.9996233978805127
RMSE второго региона: 0.893099286775616


In [15]:
#Для первого региона geo_2

target_2 = geo_data_2['product']
features_2 = geo_data_2.drop('product', axis=1)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=rnd_state)

model = LinearRegression()
model.fit(features_train_2, target_train_2)
predicted_valid_2 = model.predict(features_valid_2)
r2 = r2_score(target_valid_2, predicted_valid_2)
mse = mean_squared_error(target_valid_2, predicted_valid_2)
print("MSE третьего региона:", mse)
print("R2 третьего региона:", r2)
print("RMSE третьего региона:", mse ** 0.5)

MSE третьего региона: 1602.3775813236196
R2 третьего региона: 0.20524758386040443
RMSE третьего региона: 40.02970873393434


### Вывод:
В 1 и 3 регионах средние значение объема в месторождении примерно одинаковые, при этом модели имеют большую погрешность.

Во 2 регионе средние запасы нефти в месторождениях ниже, но и погрешность модели значительно ниже.

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
так же среднее тут было бы для полной картины кстати)<br>
    

# 3. Подготовка к расчёту прибыли

Для всех ключевых значений созданы константы Python:

In [16]:
POINTS = 500
BOOTSTRAP_SAMPLES = 1000
BUDGET = 10e9
COST = 50e6
BARREL_PROFIT = 450 #4500
RISK_DAMAGE = 0.025
CONF_INTERVAL = 0.95

Минимальное среднее количество продукта в месторождениях региона, достаточное для разработки:

In [17]:
min_barrels = COST / (BARREL_PROFIT * 1000)
min_barrels

111.11111111111111

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
есть точка безубыточности) И видно, что нужны DS-специалисты нефтянке:)<br>
(убрал один 0 в BARREL_PROFIT ) 

<font color='black'> спасибо! =)
    

Максимальное количество месторождений, которое можно разработать:

In [18]:
max_wells = int(BUDGET / COST)
max_wells

200

Функция расчета прибыли:

In [19]:
def revenue(target, probabilities, count):
    top = probabilities.sort_values(ascending=False)#.head(count)
    total_profit = (target.loc[top.index][:count] * 1000 * BARREL_PROFIT).sum()
    return total_profit - (COST * count)

### Вывод:

- Посчитали среднее количество продукта

- Максимальное количество месторождений

- Создали константы для ключевых значений

- Создали функцию прибыли

# 4. Расчёт прибыли и рисков 

### Расчёт прибыли и рисков для geo_0 (для первого региона)

In [20]:
target_valid_0 = target_valid_0.reset_index(drop=True)

In [21]:
state = np.random.RandomState(seed=12345)

values = []
for i in range(BOOTSTRAP_SAMPLES):
    target_subsample = target_valid_0.sample(POINTS, replace=True, random_state=state)
#    probs_subsample = pd.Series(data=target_subsample, index=target_subsample.index)
    probs_subsample = pd.Series(predicted_valid_0)[target_subsample.index]   
    values.append(revenue(target_subsample, probs_subsample, max_wells))

values = pd.Series(values)
risk = (values < 0).mean()
mean = values.mean()

confidence_interval = (values.quantile(0.025), values.quantile(0.975))

print("Средняя выручка:", mean)
print("Доверительный интервал:", confidence_interval)
print("Риск убытков:", risk)
#print("Квантиль 2,5%:", lower)

Средняя выручка: 425938526.91059226
Доверительный интервал: (-102090094.8379384, 947976353.358369)
Риск убытков: 0.06


### Расчёт прибыли и рисков для geo_1 (для второго региона)

In [22]:
target_valid_1 = target_valid_1.reset_index(drop=True)

In [23]:
state = np.random.RandomState(seed=12345)

values = []
for i in range(BOOTSTRAP_SAMPLES):
    target_subsample = target_valid_1.sample(POINTS, replace=True, random_state=state)
    probs_subsample = pd.Series(predicted_valid_1)[target_subsample.index]  
    values.append(revenue(target_subsample, probs_subsample, max_wells))

values = pd.Series(values)
risk = (values < 0).mean()
mean = values.mean()
#lower = values.quantile(0.025)
# confidence_interval = st.t.interval(0.95, len(values)-1, loc=values.mean(), scale=values.sem())
confidence_interval = (values.quantile(0.025), values.quantile(0.975))
print("Средняя выручка:", mean)
print("Доверительный интервал:", confidence_interval)
print("Риск убытков:", risk)
#print("Квантиль 2,5%:", lower)

Средняя выручка: 515222773.4432914
Доверительный интервал: (68873225.37050182, 931547591.2570513)
Риск убытков: 0.01


### Расчёт прибыли и рисков для geo_2 (для третьего региона)

In [24]:
target_valid_2 = target_valid_2.reset_index(drop=True)

In [25]:
state = np.random.RandomState(seed=12345)

values = []
for i in range(BOOTSTRAP_SAMPLES):
    target_subsample = target_valid_2.sample(POINTS, replace=True, random_state=state)
    probs_subsample = pd.Series(predicted_valid_2)[target_subsample.index]  
    values.append(revenue(target_subsample, probs_subsample, max_wells))


values = pd.Series(values)
risk = (values < 0).mean()
mean = values.mean()
#lower = values.quantile(0.025)
confidence_interval = (values.quantile(0.025), values.quantile(0.975))
#confidence_interval = st.t.interval(0.95, len(values)-1, loc=values.mean(), scale=values.sem())

print("Средняя выручка:", mean)
print("Доверительный интервал:", confidence_interval)
print("Риск убытков:", risk)
#print("Квантиль 2,5%:", lower)

Средняя выручка: 435008362.7827556
Доверительный интервал: (-128880547.32978901, 969706954.1802661)
Риск убытков: 0.064


### Вывод: 

Исходя из результатов и анализа расчета прибыли и рисков, наиболее продуктивным является второй регион по средней выручке, с наименьшим риском убытков, чем у остальных регионов = 0,01. Доверительный интервал при этом наиболее узкий, в сравнении с остальными регионами. 

Первый и третий регионы по прибыли и рискам очень похожи. Я бы порекомендовала выбрать второй регион. 

In [26]:
#Вывод с прошлими значениями
#Исходя из результатов расчета прибыли и рисков, 
#наиболее продуктивным регионом является - второй. 
#В котором выше средняя выручка, риск убытков (как и остальных регионов) равен нулю, 
#а доверительный интервал самый узкий в сравнении с другими регионами. 

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
Антон, на самом деле очень хорошая работа!<br>
Аргументация, удобный для просмотра и чтения код - многое удалось как надо<br>
Оформление выводов в конце - понятно и доходчиво вышло)<br>

<font color='red'>    
1)Есть пару моментов, который надо бы проверить: он касается доверительных интервалов.<br>
Cмотри, какие узкие доверительные интервалы - очень уж точная прогнозная модель выходит.<br>
Это доверительный интервала для истинного среднего.<br>
    
На мой взгляд, для бизгнеса важно выводить интервал именно для прогноза<br>
Посмотри, пожалуйста вот на этот код<br>
confidence_interval = (profit_values.quantile(0.025), profit_values.quantile(0.975))<br><br><br>

2)И ещё: вот в этих строках по каждому региону<br> probs_subsample = pd.Series(data=target_subsample, index=target_subsample.index)<br>
мне кажется должно присутствовать предсказанное значение. Посмотри, пожалуйста)

<font color='green'>    
Спасибо)<br>
Буду ждать комментариев)
<br>
</font>

Николай, привет! 

Спасибо за рекомендации и комментарии, постараюсь их учесть в будущих проектах. 

1. По первому пункту я исправил и поставил код, который ты порекоменовал. 

2. Не совсем понял, что ты имеешь ввиду вот тут: 

<font color='red'>    

2)И ещё: вот в этих строках по каждому региону<br> probs_subsample = pd.Series(data=target_subsample, index=target_subsample.index)<br>
мне кажется должно присутствовать предсказанное значение. Посмотри, пожалуйста)

<font color='black'>
Ты имеешь ввиду сделать код таким? =) 
probs_subsample = pd.Series[target_subsample.index]
    
Заранее спасибо =)

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА 2</u>
</font>
<font color='green'>
Антон, привет! Спасибо за доработку)<br>
Я подставил в регион 0 строку.<br>
Посмотри, пожалуйста.

</font>    
<font color='black'>
Николай, класс! Спасибо за помощь =)
У меня единственное вопрос (этого просто не было в тренажере), ты рекомендуешь использовать во всех случаях вот такое решение: 
1. confidence_interval = (values.quantile(0.025), values.quantile(0.975))
2. probs_subsample = pd.Series(predicted_valid_2)[target_subsample.index] 
    
???
    
Заранее спасибо за ответ)

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА 3</u>
</font>
<font color='green'>
Антон, привет! Спасибо за отклик)) <br>
Для данной задачи - точно такое) А в других - надо смотреть будет))<br>
1) Это доверительный интерва прогноза. Он показывает по сути - мин-макс возможного уровня прибыли.<br>
2) Опять же да, для этой задачи ТАК: из предсказанных выбираем проиндексированные по таргету.<br>
    
УСПЕШНОГО ДАЛЬНЕЙШЕГО ОБУЧЕНИЯ!

# Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован